In [1]:
import os
import psutil

def show_memory_info(hint):
    pid = os.getpid()
    p = psutil.Process(pid)

    info = p.memory_full_info()
    memory = info.uss / 1024. / 1024
    print('{} memory used: {} MB'.format(hint, memory))

In [3]:
def func():
    show_memory_info('init')
    a = [i for i in range(10000000)]
    show_memory_info('a created')

func()
show_memory_info('fini')

init memory used: 59.82421875 MB
a created memory used: 443.30078125 MB
fini memory used: 62.95703125 MB


In [ ]:
def func():
    global a  # 声明为全局变量，函数返回后变量不释放
    show_memory_info('init')
    a = [i for i in range(10000000)]
    show_memory_info('a created')

func()
show_memory_info('fini')

init memory used: 56.203125 MB
a created memory used: 438.55859375 MB
fini memory used: 438.55859375 MB


In [8]:
def func():
    show_memory_info('init')
    a = [i for i in range(10000000)]
    show_memory_info('a created')
    return a  # 返回变量可能增加引用计数，阻止局部对象的释放

func()  # 无实际赋值引用计数没有增加
show_memory_info('fini')

init memory used: 820.91796875 MB
a created memory used: 1205.61328125 MB
fini memory used: 823.3046875 MB


In [ ]:
def func():
    show_memory_info('init')
    a = [i for i in range(10000000)]
    show_memory_info('a created')
    return a  # 返回变量可能增加引用计数，阻止局部对象的释放

b = func()  # 实际赋值后引用计数增加
show_memory_info('fini')

init memory used: 438.42578125 MB
a created memory used: 821.39453125 MB
fini memory used: 821.39453125 MB


In [ ]:
import sys

a = []  # 变量赋值，引用计数+1
sys.getrefcount(a)  # 函数参数赋值，引用计数+1

2

In [ ]:
import sys

a = []  # 变量赋值，引用计数+1

def func(a):
    print(sys.getrefcount(a))  # 函数参数赋值，引用计数+1
    # 函数返回：参数带来的引用计数增加将会复位，应用计数-1

func(a)  # 函数参数赋值，引用计数+1
sys.getrefcount(a)

3


2

In [ ]:
import sys

a = []  # 引用计数+1

print(sys.getrefcount(a))  # 引用计数+1

b = a  # 引用计数+1

print(sys.getrefcount(a))  # 这里针对同一个对象第二次调用 sys.getrefcount 不会再增加引用计数

2
3


In [18]:
import sys

a = []  # 引用计数+1

print(sys.getrefcount(a))  # 引用计数+1

b = a  # 引用计数+1

print(sys.getrefcount(a))  # 同一个对象二次调用不增加引用计数

c = b  # 引用计数+1
d = b  # 引用计数+1
e = c  # 引用计数+1
f = e  # 引用计数+1
g = d  # 引用计数+1

print(sys.getrefcount(a))  # 同一个对象再次调用不增加引用计数

2
3
8


In [5]:
import gc

show_memory_info('init')

a = [i for i in range(1000000)]

show_memory_info('a created')

del a
gc.collect()

show_memory_info('a deleted')

init memory used: 54.62109375 MB
a created memory used: 92.41796875 MB
a deleted memory used: 54.73046875 MB


In [ ]:
print(a)  # 已经销毁

NameError: name 'a' is not defined

In [ ]:
def func():
    show_memory_info('init')
    a = [i for i in range(10000000)]
    b = [i for i in range(10000000)]
    show_memory_info('a, b created')
    a.append(b)  # 循环引用
    b.append(a)  # 循环引用
    # 函数返回，a 和 b 变量本身释放，但相应的对象由于循环引用导致引用计数不归零，对象不会释放

func()
show_memory_info('fini')

init memory used: 62.84375 MB
a, b created memory used: 828.16796875 MB
fini memory used: 828.16796875 MB


In [ ]:
import gc

def func():
    show_memory_info('init')
    a = [i for i in range(10000000)]
    b = [i for i in range(10000000)]
    show_memory_info('a, b created')
    a.append(b)  # 循环引用
    b.append(a)  # 循环引用
    # 函数返回，a 和 b 变量本身释放，但相应的对象由于循环引用导致引用计数不归零，对象不会释放

func()
show_memory_info('fini')

gc.collect()  # 强制执行 GC，gc 会通过标记清除等算法识别 '孤立对象' 并释放
show_memory_info('gc')

init memory used: 63.88671875 MB
a, b created memory used: 828.48046875 MB
fini memory used: 828.48046875 MB
gc memory used: 63.8125 MB


In [25]:
import objgraph

a = [1, 2, 3]
b = [4, 5, 6]

a.append(b)
b.append(a)

objgraph.show_refs([a, b])

Graph written to C:\Users\kduser\AppData\Local\Temp\objgraph-rhxrmpxg.dot (8 nodes)
Graph viewer (xdot) not found, generating a png instead
Image generated as C:\Users\kduser\AppData\Local\Temp\objgraph-rhxrmpxg.png


In [28]:
import objgraph

a = [1, 2, 3]
b = [4, 5, 6]

a.append(b)
b.append(a)

objgraph.show_backrefs([a, b])

Graph written to C:\Users\kduser\AppData\Local\Temp\objgraph-1z82jyff.dot (18 nodes)
Graph viewer (xdot) and image renderer (dot) not found, not doing anything else
